# Emotion Detection from Text using BERT
Fine-tune BERT on the GoEmotions dataset (CSV version from GCS) for multi-label emotion classification.


In [ ]:
!pip install transformers scikit-learn --quiet

In [ ]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments


In [ ]:
!gsutil cp -r gs://gresearch/goemotions/data/full_dataset/ .

Copying gs://gresearch/goemotions/data/full_dataset/goemotions_1.csv...
Copying gs://gresearch/goemotions/data/full_dataset/goemotions_2.csv...
Copying gs://gresearch/goemotions/data/full_dataset/goemotions_3.csv...
\ [3 files][ 40.8 MiB/ 40.8 MiB]                                                
Operation completed over 3 objects/40.8 MiB.                                     


In [ ]:
# Load and merge the three CSV files downloaded from GCS
df1 = pd.read_csv("full_dataset/goemotions_1.csv")
df2 = pd.read_csv("full_dataset/goemotions_2.csv")
df3 = pd.read_csv("full_dataset/goemotions_3.csv")
data = pd.concat([df1, df2, df3], ignore_index=True)
print(f"Total samples: {len(data)}")
data.head()

Total samples: 211225


,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,1.548084e+09,37,True,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1.546428e+09,37,False,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1.547965e+09,18,False,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1.546669e+09,2,False,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# Label names from GoEmotions
label_names = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
    'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment',
    'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism',
    'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]
num_labels = len(label_names)

In [ ]:
print(data.columns)


Index(['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id', 'example_very_unclear', 'admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral'],
      dtype='object')


In [ ]:
# Convert comma-separated string labels into multi-hot encoded vectors
def encode_labels(label_str):
    vec = [0] * num_labels
    for idx in map(int, str(label_str).split(',')):
        vec[idx] = 1
    return vec

# Các cột cảm xúc (28 nhãn)
label_names = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
    'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment',
    'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism',
    'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]
num_labels = len(label_names)

# Lấy dữ liệu văn bản và vector nhãn multi-label
texts = data['text'].tolist()
labels = data[label_names].values.tolist()


In [ ]:
# Tokenize texts
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)
input_ids = encodings['input_ids']
attention_masks = encodings['attention_mask']

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Split dataset
train_ids, val_ids, train_masks, val_masks, train_labels, val_labels = train_test_split(
    input_ids, attention_masks, labels, test_size=0.1, random_state=42
)

# Dataset class
class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx]),
            'attention_mask': torch.tensor(self.attention_masks[idx]),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float)
        }

train_dataset = EmotionDataset(train_ids, train_masks, train_labels)
val_dataset = EmotionDataset(val_ids, val_masks, val_labels)

In [ ]:
# Load model
# Load model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=num_labels,
    problem_type='multi_label_classification'
)

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch', # Changed from evaluation_strategy
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs= 0.000000000001,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train model
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
0,No log,0.707233


TrainOutput(global_step=1, training_loss=0.7226958870887756, metrics={'train_runtime': 8001.4678, 'train_samples_per_second': 0.0, 'train_steps_per_second': 0.0, 'total_flos': 1052689907712.0, 'train_loss': 0.7226958870887756, 'epoch': 8.41609156707625e-05})

In [ ]:
# Evaluate model
preds = trainer.predict(val_dataset)
pred_probs = torch.sigmoid(torch.tensor(preds.predictions)).numpy()
pred_labels = (pred_probs > 0.5).astype(int)
true_labels = np.array(val_labels)
print(classification_report(true_labels, pred_labels, target_names=label_names))

                precision    recall  f1-score   support

    admiration       0.00      0.00      0.00      1728
     amusement       0.04      0.40      0.07       942
         anger       0.04      1.00      0.08       838
     annoyance       0.06      0.03      0.04      1384
      approval       0.08      0.79      0.15      1709
        caring       0.01      0.01      0.01       566
     confusion       0.03      1.00      0.07       734
     curiosity       0.05      0.79      0.10       999
        desire       0.02      1.00      0.04       400
disappointment       0.04      0.32      0.07       842
   disapproval       0.06      0.12      0.08      1156
       disgust       0.03      0.17      0.05       539
 embarrassment       0.01      1.00      0.02       262
    excitement       0.00      0.00      0.00       550
          fear       0.01      0.86      0.03       310
     gratitude       0.05      1.00      0.10      1156
         grief       0.00      0.14      0.00  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
